In [ ]:
# !ls /datasets/cms

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_parquet('/datasets/cms/prov_svc_2019_mi.parquet')
# dtype={'HCPCS_Cd': 'string', 'Rndrng_Prvdr_Zip5': 'string','Rndrng_Prvdr_RUCA': 'float64', 'Rndrng_Prvdr_State_FIPS': 'float64', 'Tot_Srvcs': 'float64'})
# df.dtypes

In [ ]:
# missing value for Rndrng_Prvdr_Crdntls and Rndrng_Prvdr_Gndr
df = df.fillna('')

In [ ]:
def crdntls_map(crdntls):
    result = crdntls
    if (type(crdntls) == type('str')):        
        result = crdntls.replace ('.','').replace('>','') \
                        .replace('DOCTOR OF AUDIOLOGY','AUD').replace('DR  AUDIOLOGY','AUD').replace('AUDIOLOGIST','AUD') \
                        .replace('DC CHIROPRACTOR','DC').replace('DOCTOR OF CHIROPRACT','DC').replace('CHIROPRACTOR DC','DC')  \
                        .replace('DC DR OF CHIROPRACTI','DC').replace('DC DOCTOR OF CHIROPR','DC').replace('CHIROPRACTIC','DC')  \
                        .replace('D O','DO').replace('D.O.','DO').replace('D.O','DO') \
                        .replace('D P M','DPM') \
                        .replace ('M D','MD').replace ('M.D.','MD').replace ('M. D.','MD').replace('M.D','MD').replace('M>D>','MD') \
                        .replace('M D','MD').replace('M,D,','MD') \
                        .replace('N P','NP') \
                        .replace('NURSE PRACTITIONER','NP').replace('NURSE PRACITIONER','NP').replace('NP NURSE PRACTITIONE','NP') \
                        .replace('NURSE PRACTIONER','NP').replace('NP- NURSE PRACTITION', 'NP').replace('NURSE PRACTTIONER','NP') \
                        .replace('NP(WOMEN\'S HEALTH)','NP').replace('FAMILY NURSE PRACTIT','NP').replace('FAMILY NURSE PRACTIO', 'NP') \
                        .replace('NATALIE BROWN','').replace('ISLAM GOMAA','') \
                        .replace('FAMILY PRACTICE','').replace('INTERNAL MEDICINE','') \
                        .replace('OD DR OF OPTOMETRY','OD').replace('LINDSEY WYNKOOP, OD','OD')  \
                        .replace('PA-C','PAC').replace('(PA-C)','PAC').replace('(PAC)','PAC').replace('PA -C','PAC') \
                        .replace('PH D','PHD').replace('PH.D.','PHD').replace('DOCTORATE IN PHYSICA','PHD')  \
                        .replace('PHYSICIAN ASSISTANT','PA').replace('PHYSICIANS ASSISTANT','PA').replace('WILLIAM BLAKESLEE PA','PA') \
                        .replace('P A','PA').replace('PHYSICAL THERAPIST', 'PT').replace('P T','PT') \
                        .replace('REGISTERED DIETITIAN','RD')  \
                        .replace('SOCIAL WORKER','SW')
    else:
        result = ''
    
    return result

def isMD(crdntls):
    result = 0
    if (('MD' in crdntls) or ('DO' in crdntls) or ('MBBS' in crdntls)):
        result = 1
    return result

df.Rndrng_Prvdr_Crdntls = df.Rndrng_Prvdr_Crdntls.apply(lambda x: crdntls_map(x))
df['md'] = df.Rndrng_Prvdr_Crdntls.apply(lambda x: isMD(x))

In [ ]:
# len(df.Rndrng_Prvdr_Crdntls.unique()), df.Rndrng_Prvdr_Crdntls.unique()
# df.Rndrng_Prvdr_Gndr.unique()
# df.Rndrng_Prvdr_Ent_Cd.unique()
# df.Rndrng_Prvdr_Type.unique()
# df.Rndrng_Prvdr_Mdcr_Prtcptg_Ind.unique()
# df.HCPCS_Drug_Ind.unique()
# df.Place_Of_Srvc.unique()

In [ ]:
df['gender'] = np.where((df.Rndrng_Prvdr_Gndr == 'M'), 1, 0) 
df['entity'] = np.where(df.Rndrng_Prvdr_Ent_Cd == 'O', 1, 0) # organization or individual
df['Rndrng_Prvdr_Mdcr_Prtcptg_Ind'] = np.where(df.Rndrng_Prvdr_Mdcr_Prtcptg_Ind == 'Y', 1, 0)
df['HCPCS_Drug_Ind'] = np.where(df.HCPCS_Drug_Ind == 'Y', 1, 0)
df['facility'] = np.where(df.Place_Of_Srvc == 'F', 1, 0)

In [ ]:
# len(df.md.unique()), len(df.gender.unique()), len(df.entity.unique()), len(df.facility.unique())

In [ ]:
# df.Rndrng_Prvdr_RUCA.unique()
# df.groupby('Rndrng_Prvdr_RUCA').count()

df['Rndrng_Prvdr_RUCA'] = pd.to_numeric(df['Rndrng_Prvdr_RUCA'])
df['metro'] = np.where((df.Rndrng_Prvdr_RUCA <=6), 1, 0)

# df.groupby('metro').count()

In [ ]:
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('A'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('C'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('G'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('J'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('P'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('Q'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('R'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.startswith('V'), df.HCPCS_Cd.str[1:], df.HCPCS_Cd)
# df['HCPCS_Cd'] = np.where(df.HCPCS_Cd.str.endswith('T'), df.HCPCS_Cd.str[:-2], df.HCPCS_Cd)
# df['HCPCS_Cd'] = pd.to_numeric(df['HCPCS_Cd'])

In [ ]:
df = df.reset_index()
df['Tot_Benes'] = pd.to_numeric(df['Tot_Benes'])
df['Tot_Srvcs'] = pd.to_numeric(df['Tot_Srvcs'])
df['Tot_Bene_Day_Srvcs'] = pd.to_numeric(df['Tot_Bene_Day_Srvcs'])
df['Avg_Sbmtd_Chrg'] = pd.to_numeric(df['Avg_Sbmtd_Chrg'])
df['Avg_Mdcr_Alowd_Amt'] = pd.to_numeric(df['Avg_Mdcr_Alowd_Amt'])
df['Avg_Mdcr_Pymt_Amt'] = pd.to_numeric(df['Avg_Mdcr_Pymt_Amt'])
df['Avg_Mdcr_Stdzd_Amt'] = pd.to_numeric(df['Avg_Mdcr_Stdzd_Amt'])

In [ ]:
df = df.drop(columns=['index', 'Rndrng_NPI', 'Rndrng_Prvdr_Last_Org_Name', 'Rndrng_Prvdr_First_Name', 'Rndrng_Prvdr_MI', \
                'Rndrng_Prvdr_St1', 'Rndrng_Prvdr_St2', 'Rndrng_Prvdr_City', 'Rndrng_Prvdr_State_Abrvtn', \
                'Rndrng_Prvdr_State_FIPS', 'Rndrng_Prvdr_Zip5', 'Rndrng_Prvdr_RUCA_Desc', 'Rndrng_Prvdr_Cntry', \
                'Rndrng_Prvdr_Type', 'HCPCS_Cd', 'HCPCS_Desc'])

# these 2 are highly related to Avg_Mdcr_Alowd_Amt over 99%
df = df.drop(columns=['Avg_Mdcr_Pymt_Amt', 'Avg_Mdcr_Stdzd_Amt'])

# following columns converted to new binary column
df = df.drop(columns=['Rndrng_Prvdr_Crdntls', 'Rndrng_Prvdr_Gndr', 'Rndrng_Prvdr_Ent_Cd', 'Rndrng_Prvdr_RUCA', 'Place_Of_Srvc'])


In [ ]:
df.dtypes

Rndrng_Prvdr_Mdcr_Prtcptg_Ind      int64
HCPCS_Drug_Ind                     int64
Tot_Benes                          int64
Tot_Srvcs                        float64
Tot_Bene_Day_Srvcs                 int64
Avg_Sbmtd_Chrg                   float64
Avg_Mdcr_Alowd_Amt               float64
md                                 int64
gender                             int64
entity                             int64
facility                           int64
metro                              int64
dtype: object

In [ ]:
df.count()

Rndrng_Prvdr_Mdcr_Prtcptg_Ind    337351
HCPCS_Drug_Ind                   337351
Tot_Benes                        337351
Tot_Srvcs                        337351
Tot_Bene_Day_Srvcs               337351
Avg_Sbmtd_Chrg                   337351
Avg_Mdcr_Alowd_Amt               337351
md                               337351
gender                           337351
entity                           337351
facility                         337351
metro                            337351
dtype: int64

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f3594ae2-e1b0-4476-8420-ffda39dfce6e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>